In [1]:
from modules.winners_predictor import WinnersPredictor
from modules.backtest import Backtest
from datetime import datetime

import pandas as pd
import itertools

import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
model = WinnersPredictor() # tworzymy obiekt dokonujacy predykcji dla poszczegolnych dat
backtest = Backtest() # tworzymy obiekt odpowiedzialny za obliczanie statystyk do backtestu

In [3]:
thresholds = [ # progi, od ktorych przyznajemy nagrody
    [0.7, 0.8, 0.9, 0.95], 
    [0.75, 0.8, 0.85, 0.9], 
    [0.8, 0.85, 0.9, 0.95], 
    [0.9, 0.92, 0.94, 0.96], 
    [0.92, 0.94, 0.96, 0.98], 
    [0.6, 0.7, 0.8, 0.9], 
    [0.5, 0.7, 0.8, 0.9]
]

rewards = [0.25, 0.5, 0.75, 1.0] # nagrody

nums_of_indicators = [8, 10, 11, 12, 14] # liczba wskaznikow, na ktorych chcemy bazowac

same_type_items = [2,3,4] # maksymalna liczba wskaznikow tego samego typu

tmp2 = list(range(1,9))
tmp_dict = {str(x[0]): x[1] for x in list(zip(thresholds, tmp2))}

In [4]:
winners = {}
# petla, ktora leci po wszystkich kombinacjach powyzszych ustawien
# winnerzy wraz z datami są zapisywani do slownika "winners"
for num_of_indicators, max_same_type_items, threshold in itertools.product(nums_of_indicators, same_type_items, thresholds):
    threshold_num = tmp_dict[str(threshold)]
    winners_tmp = {}
    for quarter in pd.date_range(start='1998-01-01', end='2017-03-31', freq='Q'):
        insample_period = backtest.get_insample_period(quarter)
        model.fit(insample_period, max_same_type_items, num_of_indicators)
        scores, tickers = model.pre_predict(quarter, thresholds=threshold, rewards=rewards)
        strdate = datetime.strftime(quarter, '%Y-%m-%d')
        winners_tmp[strdate] = tickers
    winners[f'({num_of_indicators}, {max_same_type_items}, {threshold_num})'] = winners_tmp

In [5]:
import json
with open('tmp.json', 'w') as f:
    json.dump(winners,f)

In [6]:
wyniki = {} # slownik przechowujacy wyniki dla poszczegolnych kombinacji
for combination in winners.keys():
    print(combination)
    lista_wynikow = []
    for quarter in winners[combination]:
        performance = backtest.calculate_performance(quarter, winners[combination][quarter])
        lista_wynikow.append(performance)
        wyniki[combination] = lista_wynikow

(8, 2, 1)
(8, 2, 2)
(8, 2, 3)
(8, 2, 4)
(8, 2, 5)
(8, 2, 6)
(8, 2, 7)
(8, 3, 1)
(8, 3, 2)
(8, 3, 3)
(8, 3, 4)
(8, 3, 5)
(8, 3, 6)
(8, 3, 7)
(8, 4, 1)
(8, 4, 2)
(8, 4, 3)
(8, 4, 4)
(8, 4, 5)
(8, 4, 6)
(8, 4, 7)
(10, 2, 1)
(10, 2, 2)
(10, 2, 3)
(10, 2, 4)
(10, 2, 5)
(10, 2, 6)
(10, 2, 7)
(10, 3, 1)
(10, 3, 2)
(10, 3, 3)
(10, 3, 4)
(10, 3, 5)
(10, 3, 6)
(10, 3, 7)
(10, 4, 1)
(10, 4, 2)
(10, 4, 3)
(10, 4, 4)
(10, 4, 5)
(10, 4, 6)
(10, 4, 7)
(11, 2, 1)
(11, 2, 2)
(11, 2, 3)
(11, 2, 4)
(11, 2, 5)
(11, 2, 6)
(11, 2, 7)
(11, 3, 1)
(11, 3, 2)
(11, 3, 3)
(11, 3, 4)
(11, 3, 5)
(11, 3, 6)
(11, 3, 7)
(11, 4, 1)
(11, 4, 2)
(11, 4, 3)
(11, 4, 4)
(11, 4, 5)
(11, 4, 6)
(11, 4, 7)
(12, 2, 1)
(12, 2, 2)
(12, 2, 3)
(12, 2, 4)
(12, 2, 5)
(12, 2, 6)
(12, 2, 7)
(12, 3, 1)
(12, 3, 2)
(12, 3, 3)
(12, 3, 4)
(12, 3, 5)
(12, 3, 6)
(12, 3, 7)
(12, 4, 1)
(12, 4, 2)
(12, 4, 3)
(12, 4, 4)
(12, 4, 5)
(12, 4, 6)
(12, 4, 7)
(14, 2, 1)
(14, 2, 2)
(14, 2, 3)
(14, 2, 4)
(14, 2, 5)
(14, 2, 6)
(14, 2, 7)
(14, 3, 1)
(14, 3, 2

In [7]:
import numpy as np
srednie_wyniki = {} # obliczanie statystyk do backtestu
for combination in wyniki.keys():
    tmp_combination = wyniki[combination]
    portfolio_returns = np.array([x.portfolio_return for x in tmp_combination]).mean()
    max_portfolio_return = np.array([x.portfolio_return for x in tmp_combination]).max()
    min_portfolio_return = np.array([x.portfolio_return for x in tmp_combination]).min()
    spx_returns = np.array([x.spx_return for x in tmp_combination]).mean()
    portfolio_drawdown = np.array([x.portfolio_drawdown for x in tmp_combination]).mean()
    spx_drawdowns = np.array([x.spx_drawdown for x in tmp_combination]).mean()
    portfolio_spx_drawdowns = np.array([x.portfolio_spx_drawdown for x in tmp_combination]).mean()
    alpha = np.array([x.alpha for x in tmp_combination]).mean()
    min_alpha = np.array([x.alpha for x in tmp_combination]).min()
    num_of_quarters_weak_performance = len(list(filter(lambda x: x < 0, [x.alpha for x in tmp_combination])))
    sharpe = alpha/np.std(np.array([x.alpha for x in tmp_combination]))
    srednie_wyniki[combination] = {
        'avg_portfolio_return': portfolio_returns,
        'max_portfolio_return': max_portfolio_return,
        'min_portfolio_return': min_portfolio_return,
        'avg_spx_return': spx_returns,
        'avg_portfolio_drawdown': portfolio_drawdown,
        'avg_spx_drawdown': spx_drawdowns,
        'avg_portfolio_spx_drawdowns': portfolio_spx_drawdowns,
        'avg_alpha': alpha,
        'min_alpha': min_alpha,
        'alphadiff': (alpha - min_alpha),
        'alpha<0': num_of_quarters_weak_performance,
        'sharpe': sharpe
    }

In [8]:
results = pd.DataFrame(srednie_wyniki).T.sort_values(by='sharpe', ascending=False)
results.to_csv('results1.csv')

In [9]:
from ast import literal_eval
tmp = pd.read_csv('results1.csv', index_col = 'Unnamed: 0')
top_combinations = tmp.sort_values(by='sharpe', ascending=False).iloc[:10].index
top_combinations = [literal_eval(x) for x in top_combinations]
top_combinations = [[x[1], x[0], thresholds[x[2]-1]] for x in top_combinations]

In [10]:
from json import dump
with open('stats.json', 'w') as f:
    dump({"stats": top_combinations}, f, indent=4)

In [11]:
results

,avg_portfolio_return,max_portfolio_return,min_portfolio_return,avg_spx_return,avg_portfolio_drawdown,avg_spx_drawdown,avg_portfolio_spx_drawdowns,avg_alpha,min_alpha,alphadiff,alpha<0,sharpe
"(10, 2, 7)",1.900402,4.391033,-0.215662,0.523264,-0.326348,-0.34129,-0.085798,1.377137,-0.087266,1.464403,2.0,1.677216
"(10, 2, 2)",1.851702,4.391033,-0.215662,0.523264,-0.320828,-0.34129,-0.094450,1.328438,-0.098702,1.427140,2.0,1.666648
"(11, 2, 2)",1.863184,4.415646,-0.215662,0.523264,-0.324718,-0.34129,-0.086050,1.339920,-0.087266,1.427185,1.0,1.646043
"(12, 2, 2)",1.838709,4.415193,-0.050484,0.523264,-0.322009,-0.34129,-0.090792,1.315444,0.042135,1.273310,0.0,1.629662
"(10, 2, 6)",1.848862,4.391033,-0.215662,0.523264,-0.326488,-0.34129,-0.088554,1.325598,-0.087266,1.412864,3.0,1.628848
...,...,...,...,...,...,...,...,...,...,...,...,...
"(8, 4, 2)",1.823503,5.430857,-0.145025,0.523264,-0.309295,-0.34129,-0.085934,1.300238,0.015353,1.284885,0.0,1.222672
"(8, 3, 2)",1.823503,5.430857,-0.145025,0.523264,-0.309295,-0.34129,-0.085934,1.300238,0.015353,1.284885,0.0,1.222672
"(11, 3, 5)",1.637167,4.537477,-0.057250,0.523264,-0.292745,-0.34129,-0.100459,1.113902,-0.197348,1.311251,3.0,1.177448
"(11, 4, 5)",1.627638,5.207075,0.149540,0.523264,-0.295966,-0.34129,-0.105076,1.104374,-0.412277,1.516651,3.0,1.137367


In [12]:
tmp = pd.read_csv('results.csv', index_col = 'Unnamed: 0')
tmp

,avg_portfolio_return,max_portfolio_return,min_portfolio_return,avg_spx_return,avg_portfolio_drawdown,avg_spx_drawdown,avg_portfolio_spx_drawdowns,avg_alpha,min_alpha,alphadiff,alpha<0,sharpe
"(11, 2, 7)",2.202016,5.209425,-0.359353,0.523264,-0.342098,-0.34129,-0.105097,1.678752,-0.230957,1.909709,3.0,1.628565
"(11, 2, 6)",2.230707,5.664371,-0.396002,0.523264,-0.342459,-0.34129,-0.104441,1.707442,-0.428193,2.135635,3.0,1.581072
"(8, 2, 7)",2.089727,5.154362,-0.472912,0.523264,-0.350202,-0.34129,-0.095052,1.566462,-0.344516,1.910978,3.0,1.576231
"(14, 2, 2)",2.391994,5.437232,-0.449943,0.523264,-0.351291,-0.34129,-0.102003,1.868730,-0.482135,2.350865,3.0,1.572495
"(14, 4, 6)",2.278867,6.279687,-0.378639,0.523264,-0.332732,-0.34129,-0.094289,1.755603,-0.250243,2.005846,3.0,1.571880
...,...,...,...,...,...,...,...,...,...,...,...,...
"(11, 4, 4)",2.079806,7.352115,-0.362705,0.523264,-0.358190,-0.34129,-0.123481,1.556542,-0.241473,1.798015,4.0,1.118549
"(10, 3, 5)",1.853436,6.642386,-0.196088,0.523264,-0.353297,-0.34129,-0.148482,1.330171,-0.790654,2.120826,11.0,1.101705
"(8, 4, 4)",1.928038,7.319225,-0.162972,0.523264,-0.358352,-0.34129,-0.140494,1.404773,-0.501558,1.906331,7.0,1.079424
"(8, 3, 4)",1.913753,7.319225,-0.162972,0.523264,-0.350194,-0.34129,-0.128812,1.390488,-0.501558,1.892046,7.0,1.074139
